# Day 1 - Lab 2: Generating a Product Requirements Document (PRD) (Solution)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Introduction:**
This solution notebook demonstrates how to synthesize detailed, low-level requirements (user stories) into a high-level planning document (the PRD). It also introduces the advanced concept of using code (Pydantic models) to define and validate the structure of documentation.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
We begin by setting up our environment and loading the key artifact from Lab 1: `day1_user_stories.json`. The `load_artifact` helper function reads the file content, and `json.loads` parses the JSON string into a Python list of dictionaries, making it ready for use in our prompts.

In [1]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, recommended_models_table, prompt_enhancer

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the artifact from Lab 1
user_stories_str = load_artifact("artifacts/day1_user_stories.json")
if user_stories_str:
    user_stories_data = json.loads(user_stories_str)
else:
    print("Warning: Could not load user stories. Lab may not function correctly.")
    user_stories_data = []

✅ LLM Client configured: Using 'google' with model 'gemini-2.5-pro'


In [2]:
recommended_models_table()

| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |
|---|---|---|---|---|---|---|---|---|
| Qwen/Qwen-Image | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| Qwen/Qwen-Image-Edit | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |
| black-forest-labs/FLUX.1-Kontext-dev | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |
| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| dall-e-3 | openai | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |
| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |
| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |
| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |
| gemini-2.0-flash-preview-image-generation | google | ❌ | ❌ | ✅ | ❌ | ❌ | 32,000 | 8,192 |
| gemini-2.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-flash-image-preview | google | ❌ | ❌ | ✅ | ❌ | ❌ | 32,768 | 32,768 |
| gemini-2.5-flash-lite | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-2.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 65,536 |
| gemini-live-2.5-flash-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |
| gpt-4.1 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,768 |
| gpt-4.1-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4.1-nano | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 32,000 |
| gpt-4o | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-4o-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 128,000 | 16,384 |
| gpt-5-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-mini-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| gpt-5-nano-2025-08-07 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 400,000 | 128,000 |
| meta-llama/Llama-3.3-70B-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 8,192 | 4,096 |
| meta-llama/Llama-4-Maverick-17B-128E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |
| meta-llama/Llama-4-Scout-17B-16E-Instruct | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 10,000,000 | 100,000 |
| mistralai/Mistral-7B-Instruct-v0.3 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 32,768 | 8,192 |
| o3 | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| o4-mini | openai | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |
| stabilityai/stable-diffusion-3.5-large | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |
| tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.5 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 4,096 | 1,024 |
| veo-3.0-fast-generate-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,024 | - |
| veo-3.0-generate-preview | google | ❌ | ❌ | ❌ | ❌ | ❌ | 1,024 | - |
| whisper-1 | openai | ❌ | ❌ | ❌ | ❌ | ✅ | - | - |

'| Model | Provider | Text | Vision | Image Gen | Image Edit | Audio Transcription | Context Window | Max Output Tokens |\n|---|---|---|---|---|---|---|---|---|\n| Qwen/Qwen-Image | huggingface | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |\n| Qwen/Qwen-Image-Edit | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |\n| black-forest-labs/FLUX.1-Kontext-dev | huggingface | ❌ | ❌ | ❌ | ✅ | ❌ | - | - |\n| claude-opus-4-1-20250805 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-opus-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 200,000 | 100,000 |\n| claude-sonnet-4-20250514 | anthropic | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 100,000 |\n| dall-e-3 | openai | ❌ | ❌ | ✅ | ❌ | ❌ | - | - |\n| deepseek-ai/DeepSeek-V3.1 | huggingface | ✅ | ❌ | ❌ | ❌ | ❌ | 128,000 | 100,000 |\n| gemini-1.5-flash | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,000,000 | 8,192 |\n| gemini-1.5-pro | google | ✅ | ✅ | ❌ | ❌ | ❌ | 2,000,000 | 8,192 |\n| gemini-2.0-flash-exp | google | ✅ | ✅ | ❌ | ❌ | ❌ | 1,048,576 | 8,192 |\n| gemini-2.0-flash-preview

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a Simple PRD

**Explanation:**
This prompt serves as a baseline. We provide the LLM with the user stories and ask it to summarize them into a few key sections. This is a simple synthesis task that demonstrates the LLM's ability to extract and reorganize information.

In [3]:
simple_prd_prompt = f"""
You are a Product Manager writing a Product Requirements Document (PRD) for a new hire onboarding tool.

Use the following JSON data containing user stories as your primary source of information:
<user_stories>
{user_stories_str}
</user_stories>

Generate a PRD in markdown format with the following sections:
1. **Introduction:** A brief overview of the project's purpose.
2. **User Personas:** A summary of the key users involved.
3. **Features / User Stories:** A list of the user stories and their acceptance criteria.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    # Enhance the prompt to improve structure and consistency
    enhanced_simple_prd_prompt = prompt_enhancer(simple_prd_prompt)
    print("Enhanced Simple PRD prompt:\n", enhanced_simple_prd_prompt)
    simple_prd_output = get_completion(enhanced_simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
✅ LLM Client configured: Using 'openai' with model 'o3'
Enhanced Simple PRD prompt:
 <prompt>

  <persona>
    You are a Senior Product Manager and expert Technical Writer specializing in SaaS onboarding tools.
  </persona>

  <context>
    Background:
    • Your task is to create a Product Requirements Document (PRD) for a new-hire onboarding platform.  
    • Use the following JSON array of user stories as the sole factual source. Do not invent additional stories or acceptance criteria.

    ```json
    [
      {
        "id": 1,
        "persona": "Chloe the New Hire Engineer",
        "user_story": "As a Chloe the New Hire Engineer, I want to access a Personalized Welcome Hub, so that I can quickly find all the resources I need to get started.",
        "acceptance_criteria": [
          "Given I have logged into the onboarding platform, When I navigate to the Welcome Hub, Then I see a personalized dashboard with links to key resources.",
          "Gi

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Explanation:**
This is a more advanced and practical task. Providing a template gives us much greater control over the final output. The LLM's task shifts from creative writing to structured content generation. We instruct it to fill in every section, which forces it to infer logical content for sections like "Success Metrics" and "Out of Scope" based on the provided requirements. This is a powerful pattern for creating consistent documentation.

In [4]:
# Load the PRD template from the 'templates' directory.
prd_template_content = load_artifact("templates/prd_template.md")

# If load_artifact failed to return content, try reading the file directly from disk using project_root.
if not prd_template_content:
    template_path = os.path.join(project_root, "templates", "prd_template.md")
    if os.path.exists(template_path):
        try:
            with open(template_path, "r", encoding="utf-8") as f:
                prd_template_content = f.read()
            print(f"✅ Loaded PRD template directly from {template_path} ({len(prd_template_content)} characters)")
        except Exception as e:
            print("❌ ERROR: Failed to read template file directly:", e)
            print("   Current working directory:", os.getcwd())
            prd_template_content = ""
    else:
        print("❌ ERROR: Could not find PRD template at expected path:", template_path)
        print("   Please ensure the file exists and is readable.")
        print("   Current working directory:", os.getcwd())
        prd_template_content = ""
else:
    print(f"✅ Successfully loaded PRD template ({len(prd_template_content)} characters)")

# Build the template-based prompt
template_prd_prompt = f"""
You are a Senior Product Manager responsible for creating a detailed and formal Product Requirements Document (PRD).

Your task is to populate the provided PRD template using the information from the user stories JSON.

<prd_template>
{prd_template_content}
</prd_template>

<user_stories_json>
{user_stories_str}
</user_stories_json>

Fill out every section of the template. For sections like 'Success Metrics' or 'Out of Scope', you must infer reasonable content based on the user stories and the overall project goal of creating a new hire onboarding tool.
The final output should be the completed PRD in markdown format.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    # Enhance the template-based prompt to improve fidelity and structure
    enhanced_template_prd_prompt = prompt_enhancer(template_prd_prompt)
    print("Enhanced Template PRD prompt:\n", enhanced_template_prd_prompt)
    prd_from_template_output = get_completion(enhanced_template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    if not user_stories_data:
        print("   - User stories data is missing")
    if not prd_template_content:
        print("   - PRD template is missing")
    prd_from_template_output = ""

❌ Error: Artifact file not found at templates/prd_template.md.
✅ Loaded PRD template directly from /Users/armando/Documents/Github/AG-AISOFTDEV/templates/prd_template.md (4217 characters)
--- Generating PRD from Template ---
✅ LLM Client configured: Using 'openai' with model 'o3'
Enhanced Template PRD prompt:
 <prompt>

  <persona>
    You are an experienced Senior Product Manager and Technical Writer specializing in B2B SaaS HR solutions.  
    Your job is to synthesize business requirements into clear, comprehensive Product Requirements Documents (PRDs).
  </persona>

  <context>
    We are developing a web-based New-Hire Onboarding Platform for mid-size technology companies.  
    You are provided with:
      • A PRD markdown template that contains section headings, guidance text, and example tables.  
      • A JSON array of Agile user stories with IDs, personas, stories, and acceptance criteria.  
    Goal: Fully populate the PRD template using ONLY the information in the user-sto

In [5]:
# Diagnostic: Check template file existence and path resolution
print("=== TEMPLATE DIAGNOSTIC ===")
print("Current working directory:", os.getcwd())

# Check if templates directory exists
templates_dir = os.path.join(project_root, "templates")
print(f"Project root: {project_root}")
print(f"Templates directory: {templates_dir}")
print(f"Templates directory exists: {os.path.exists(templates_dir)}")

if os.path.exists(templates_dir):
    print("Contents of templates directory:")
    for item in os.listdir(templates_dir):
        full_path = os.path.join(templates_dir, item)
        print(f"  - {item} ({'file' if os.path.isfile(full_path) else 'directory'})")

# Check the specific template file
template_file = os.path.join(project_root, "templates", "prd_template.md")
print(f"\nPRD template file path: {template_file}")
print(f"PRD template file exists: {os.path.exists(template_file)}")

if os.path.exists(template_file):
    print(f"File size: {os.path.getsize(template_file)} bytes")
else:
    print("❌ Template file not found!")
    
print("==========================")

=== TEMPLATE DIAGNOSTIC ===
Current working directory: /Users/armando/Documents/Github/AG-AISOFTDEV/Solutions/Day_01_Planning_and_Requirements
Project root: /Users/armando/Documents/Github/AG-AISOFTDEV
Templates directory: /Users/armando/Documents/Github/AG-AISOFTDEV/templates
Templates directory exists: True
Contents of templates directory:
  - adr_template.md (file)
  - prd_template.md (file)

PRD template file path: /Users/armando/Documents/Github/AG-AISOFTDEV/templates/prd_template.md
PRD template file exists: True
File size: 4217 bytes


### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Explanation:**
This is the most advanced challenge. We are now using an LLM to write *code that validates documents*. Generating a Pydantic model turns our document's structure into a testable, code-based standard. This is a form of 'documentation-as-code' that allows for automated governance, ensuring all future PRDs conform to the same reliable format.

1.  **Prompting for Code:** We give the LLM the PRD template and ask it to generate a Pydantic model. Pydantic is a data validation library, and using it to define our document structure turns that structure into a testable, reusable standard.
2.  **Saving the Model:** We save the generated Python code to a specific location (`app/validation_models/prd_model.py`). This isn't just a temporary script; it's a formal part of our application's codebase, intended to be used for future validation tasks.
3.  **Saving the PRD:** Finally, we save the markdown PRD generated in the intermediate step. This becomes the official `day1_prd.md` artifact for our project.

In [6]:
pydantic_model_prompt = f"""
You are a Python developer specializing in data validation with Pydantic.

Based on the following markdown PRD template, generate a single Pydantic model class named `ProductRequirementsDocument` that represents its structure.

<prd_template>
{prd_template_content}
</prd_template>

The model should have fields that correspond to the main sections of the template. Use appropriate Python types (e.g., str, List, Dict) from the `typing` library.
Ensure you include the necessary imports from `pydantic` and `typing`.
Only output the raw Python code for the model, without any explanation.
"""

print("--- Generating Pydantic Model for PRD ---")

if prd_template_content:
    # Enhance the prompt to encourage precise, import-ready Python code
    enhanced_pydantic_model_prompt = prompt_enhancer(pydantic_model_prompt)
    print("Enhanced Pydantic Model prompt:\n", enhanced_pydantic_model_prompt)
    pydantic_model_code = get_completion(enhanced_pydantic_model_prompt, client, model_name, api_provider)
    
    if pydantic_model_code:
        # Use our standardized cleaning function
        cleaned_code = clean_llm_output(pydantic_model_code, language='python')
        
        print("\n--- Generated Pydantic Model ---")
        print(cleaned_code)

        # Save the generated Pydantic model code to a file.
        model_path = "app/validation_models/prd_model.py"
        save_artifact(cleaned_code, model_path)
    else:
        print("Warning: Pydantic model generation failed, get_completion returned None.")
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge as our official artifact
if prd_from_template_output:
    save_artifact(prd_from_template_output, "artifacts/day1_prd.md")

--- Generating Pydantic Model for PRD ---
✅ LLM Client configured: Using 'openai' with model 'o3'
Enhanced Pydantic Model prompt:
 <prompt>
    <persona>
You are a senior Python developer with expert-level knowledge of data modeling and validation using the Pydantic library.
    </persona>

    <context>
You will be given a Markdown Product Requirements Document (PRD) template that defines the canonical structure of a PRD.  
Your task is to convert that template into a single Pydantic model class whose fields mirror the template’s main sections so the document can be parsed and validated programmatically.
    </context>

    <prd_template>
# Product Requirements Document: [Product Name]

| Status | **Draft** |
| :--- | :--- |
| **Author** | [Your Name / Team Name] |
| **Version** | 1.0 |
| **Last Updated** | [Date] |

## 1. Executive Summary & Vision
*A high-level overview for stakeholders. What is this product, why are we building it, and what is the ultimate vision for its success?*


## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.